In [1]:
class UnionFindSet: 
    def __init__(self, n):
        self._parents = [i for i in range(n + 1)]
        self._ranks = [1 for i in range(n + 1)]
        
    def find(self, u):
        while u != self._parents[u]:
            self.parents[u] = self._parents[self._parents[u]]
            u = self._parents[u]
        return u 
    
    def union(self, u, v):
        pu, pv = self.find(u), self.find(v)
        if pu == pv:
            return False         
        if self._ranks[pu] < self._ranks[pv]:
            self._parents[pu] = pv
        elif self._ranks[pu] > self._ranks[pv]:
            self._parents[pv] = pu
        else:
            self._parents[pv] = pu
            self._ranks[pu] += 1
        return True 

## 684. Redundant Connection 

In this problem, a tree is an undirected graph that is connected and has no cycles. 

The given input is a graph that started as a tree with N nodes (with distinct values 1, 2, ..., N), with one additional edge added. The added edge has two different vertices chosen from 1 to N, and was not an edge that already existed. 

The resulting graph is given as a 2D-array of edges. Each element of edges is a pair [u, v] with u < v, that represents an undirected edge connecting nodes u and v. 

Return an edge that can be removed so that the resulting graph is a tree of N nodes. If there are multiple answers, return the answer that occurs last in the given 2D-array. The answer edge [u,v] should be in the same format, with u < v. 

**Example 1:**

Input: [[1,2], [1,3], [2,3]]

Output: [2,3]

**Example 2:**

Input: [[1,2], [2,3], [3,4], [1,4], [1,5]]

Output: [1,4]



In [ ]:
def findRedundantConnection(edges: list) -> list:
    
    